# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [77]:
import numpy as np
import os
from scipy.misc import imread, imresize
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [78]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.set_random_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [79]:
train_doc = np.random.permutation(open('Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('Project_data/val.csv').readlines())
batch_size = 40 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [80]:
y = 120   # y 
z = 120   # z 
x = 30    #FRAMES

In [81]:
#data resizing - Resizing all the images, so we can have all the images in a specific size
def crop_resize_img(img):
    if img.shape[0] != img.shape[1]:
        img=img[0:120,10:150]
    resized_image = imresize(img, (y,z))
    return resized_image

In [82]:

def normalize_image(img):
    #using percentile for normalization for images, as min-max is giving better results.
    # normalized_image= img/256 
    normalized_image= img/256
    return normalized_image

In [83]:
def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)]  #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list) // batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    resized_image=crop_resize_img(image)
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = normalize_image(resized_image[:, : , 0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = normalize_image(resized_image[:, : , 1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = normalize_image(resized_image[:, : , 2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield

Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [84]:
curr_dt_time = datetime.datetime.now()
train_path = 'Project_data/train'
val_path = 'Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 20


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [1]:
3D convolution model

SyntaxError: invalid syntax (<ipython-input-1-d93c2dbb3a27>, line 1)

In [85]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers

#write your model here
# Parameters initialization
nb_rows = 120   # X dimension of the image
nb_cols = 120   # Y dimesnion of the image
#total_frames = 30
nb_frames =  30  # lenght of the video frames
nb_channel = 3 # numbe rof channels in images 3 for color(RGB) and 1 for Gray

nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(nb_frames,nb_rows,nb_cols,nb_channel)

# Define model
model = Sequential()

model.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(BatchNormalization())
model.add(Activation('relu'))

model.add(MaxPooling3D(pool_size=(2,2,2)))

model.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model.add(Flatten())

model.add(Dense(nb_dense[0], activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_dense[1], activation='relu'))
model.add(Dropout(0.5))

#softmax layer
model.add(Dense(nb_dense[2], activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [86]:
from keras.optimizers import Adam
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_5 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_46 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_23 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_5 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_47 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_24 (Activation)   (None, 15, 60, 60, 16)    0         
__________

Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [87]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [88]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [89]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [90]:
batch_size = 40

num_epochs = 21
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 40
Source path =  Project_data/train ; batch size = 40
Epoch 1/21


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 1/17 [>.............................] - ETA: 2:53 - loss: 3.1907 - categorical_accuracy: 0.3250

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 85s 5s/step - loss: 1.8970 - categorical_accuracy: 0.2634 - val_loss: 1.4946 - val_categorical_accuracy: 0.3100

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-1.90177-0.26244-1.49463-0.31000.h5
Epoch 2/21
17/17 [==============================] - 43s 3s/step - loss: 1.5275 - categorical_accuracy: 0.2916 - val_loss: 1.5198 - val_categorical_accuracy: 0.2500

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-1.52745-0.29156-1.51982-0.25000.h5
Epoch 3/21
17/17 [==============================] - 42s 2s/step - loss: 1.4311 - categorical_accuracy: 0.3720 - val_loss: 1.2452 - val_categorical_accuracy: 0.5500

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-1.43320-0.37330-1.24516-0.55000.h5
Epoch 4/21
17/17 [==============================] - 39s 2s/step - loss: 1.3254 - categorical_accuracy: 0.3808 - val_loss: 1.1883 - val_categorical_accuracy: 0.4833

Epoch 00004:

In [91]:
batch_size = 50
num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


17/17 [==============================] - 39s 2s/step - loss: 0.5952 - categorical_accuracy: 0.7820 - val_loss: 0.7058 - val_categorical_accuracy: 0.7500

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-0.59521-0.78201-0.70580-0.75000.h5
Epoch 2/10
17/17 [==============================] - 33s 2s/step - loss: 0.5701 - categorical_accuracy: 0.7958 - val_loss: 0.6190 - val_categorical_accuracy: 0.8167

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-0.57007-0.79585-0.61903-0.81667.h5
Epoch 3/10
17/17 [==============================] - 34s 2s/step - loss: 0.6353 - categorical_accuracy: 0.7439 - val_loss: 0.7683 - val_categorical_accuracy: 0.7167

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-0.63534-0.74394-0.76832-0.71667.h5
Epoch 4/10
17/17 [==============================] - 34s 2s/step - loss: 0.5865 - categorical_accuracy: 0.7543 - val_loss: 0.5563 - val_categorical_accuracy: 0.8167

Epoch 00004:

In [93]:
batch_size = 50
num_epochs = 20

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/train ; batch size = 50
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


12/17 [====================>.........] - ETA: 34s - loss: 0.6069 - categorical_accuracy: 0.7467

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 95s 6s/step - loss: 0.6913 - categorical_accuracy: 0.7011 - val_loss: 0.6550 - val_categorical_accuracy: 0.7933

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-0.63609-0.72934-0.65503-0.79333.h5
Epoch 2/20
17/17 [==============================] - 30s 2s/step - loss: 0.6227 - categorical_accuracy: 0.7783 - val_loss: 0.6816 - val_categorical_accuracy: 0.7667

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-0.62268-0.77828-0.68157-0.76667.h5
Epoch 3/20
17/17 [==============================] - 30s 2s/step - loss: 0.6218 - categorical_accuracy: 0.7557 - val_loss: 0.6645 - val_categorical_accuracy: 0.7867

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-0.62182-0.75566-0.66450-0.78667.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.051757957450718e-08.
Epoch 4/20
17/17 [==============================] - 29s 2s/step - loss: 0.6716 - categorical_acc

In [94]:
batch_size = 55
num_epochs = 10

train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 55
Source path =  Project_data/train ; batch size = 55
Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 85s 5s/step - loss: 0.6959 - categorical_accuracy: 0.7081 - val_loss: 1.2257 - val_categorical_accuracy: 0.5862

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-0.62464-0.77037-1.22572-0.58621.h5
Epoch 2/10
17/17 [==============================] - 6s 327ms/step - loss: 0.8236 - categorical_accuracy: 0.6078 - val_loss: 0.7406 - val_categorical_accuracy: 0.7385

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-0.82358-0.60784-0.74060-0.73846.h5
Epoch 3/10
17/17 [==============================] - 7s 393ms/step - loss: 0.8339 - categorical_accuracy: 0.6667 - val_loss: 0.6259 - val_categorical_accuracy: 0.8333

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-0.83392-0.66667-0.62589-0.83333.h5
Epoch 4/10
17/17 [==============================] - 7s 417ms/step - loss: 0.8292 - categorical_accuracy: 0.6667 - val_loss: 0.6562 - val_categorical_accuracy: 0.8000

Epoch 

In [95]:
batch_size = 55
num_epochs = 20
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 55
Source path =  Project_data/train ; batch size = 55
Epoch 1/20


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 85s 5s/step - loss: 0.5725 - categorical_accuracy: 0.8011 - val_loss: 2.0188 - val_categorical_accuracy: 0.6414

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-0.56894-0.80148-2.01879-0.64138.h5
Epoch 2/20
17/17 [==============================] - 6s 331ms/step - loss: 0.6952 - categorical_accuracy: 0.7647 - val_loss: 0.9401 - val_categorical_accuracy: 0.6154

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-0.69523-0.76471-0.94014-0.61538.h5
Epoch 3/20
17/17 [==============================] - 7s 385ms/step - loss: 0.6801 - categorical_accuracy: 0.8039 - val_loss: 0.7417 - val_categorical_accuracy: 0.7333

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-0.68007-0.80392-0.74173-0.73333.h5
Epoch 4/20
17/17 [==============================] - 6s 376ms/step - loss: 0.6748 - categorical_accuracy: 0.7255 - val_loss: 0.5451 - val_categorical_accuracy: 0.8333

Epoch 

In [96]:
batch_size = 50
num_epochs = 30
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val ; batch size = 50
Source path =  Project_data/trainEpoch 1/30 ; batch size = 50



/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


12/17 [====================>.........] - ETA: 34s - loss: 0.5649 - categorical_accuracy: 0.7700

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 94s 6s/step - loss: 0.7390 - categorical_accuracy: 0.7301 - val_loss: 0.7075 - val_categorical_accuracy: 0.7667

Epoch 00001: saving model to model_init_2020-08-1715_59_41.119511/model-00001-0.62325-0.75641-0.70750-0.76667.h5
Epoch 2/30
17/17 [==============================] - 30s 2s/step - loss: 0.6108 - categorical_accuracy: 0.7738 - val_loss: 0.6298 - val_categorical_accuracy: 0.7933

Epoch 00002: saving model to model_init_2020-08-1715_59_41.119511/model-00002-0.61085-0.77376-0.62975-0.79333.h5
Epoch 3/30
17/17 [==============================] - 31s 2s/step - loss: 0.5952 - categorical_accuracy: 0.7421 - val_loss: 0.6488 - val_categorical_accuracy: 0.7867

Epoch 00003: saving model to model_init_2020-08-1715_59_41.119511/model-00003-0.59518-0.74208-0.64876-0.78667.h5
Epoch 4/30
17/17 [==============================] - 28s 2s/step - loss: 0.6116 - categorical_accuracy: 0.7647 - val_loss: 0.6851 - val_categorical_accuracy: 0.7733

Epoch 00004:

# CNN+RNN

In [98]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers


In [99]:
lstm_cells=64
dense_neurons=64
dropout=0.25
num_classes

model = Sequential()

model.add(TimeDistributed(Conv2D(16, (3, 3) , padding='same', activation='relu'),
                                  input_shape=(30,120,120,3)))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(32, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(64, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        
model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))

model.add(TimeDistributed(Conv2D(128, (3, 3) , padding='same', activation='relu')))
model.add(TimeDistributed(BatchNormalization()))
model.add(TimeDistributed(MaxPooling2D((2, 2))))
        

    
model.add(TimeDistributed(Flatten()))


model.add(GRU(lstm_cells))
model.add(Dropout(dropout))
        
model.add(Dense(dense_neurons,activation='relu'))
model.add(Dropout(dropout))
        
model.add(Dense(num_classes, activation='softmax'))

In [100]:
from keras.optimizers import Adam
optimiser = Adam() #write your optimizer
model.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed_129 (TimeDi (None, 30, 120, 120, 16)  448       
_________________________________________________________________
time_distributed_130 (TimeDi (None, 30, 120, 120, 16)  64        
_________________________________________________________________
time_distributed_131 (TimeDi (None, 30, 60, 60, 16)    0         
_________________________________________________________________
time_distributed_132 (TimeDi (None, 30, 60, 60, 32)    4640      
_________________________________________________________________
time_distributed_133 (TimeDi (None, 30, 60, 60, 32)    128       
_________________________________________________________________
time_distributed_134 (TimeDi (None, 30, 30, 30, 32)    0         
_________________________________________________________________
time_distributed_135 (TimeDi (None, 30, 30, 30, 64)    18496     
__________

In [101]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [102]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1)# write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [103]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [56]:
batch_size = 40

num_epochs = 20
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  Project_data/val Source path =  Project_data/train ; batch size = 40
Epoch 1/20
; batch size = 40


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


 2/17 [==>...........................] - ETA: 1:41 - loss: 1.7333 - categorical_accuracy: 0.2375

/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:34: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.


17/17 [==============================] - 85s 5s/step - loss: 1.4891 - categorical_accuracy: 0.3606 - val_loss: 1.4391 - val_categorical_accuracy: 0.3700

Epoch 00001: saving model to model_init_2020-08-1714_29_28.508381/model-00001-1.47940-0.36199-1.43912-0.37000.h5
Epoch 2/20
17/17 [==============================] - 44s 3s/step - loss: 1.2324 - categorical_accuracy: 0.4680 - val_loss: 1.1211 - val_categorical_accuracy: 0.5667

Epoch 00002: saving model to model_init_2020-08-1714_29_28.508381/model-00002-1.23242-0.46803-1.12105-0.56667.h5
Epoch 3/20
17/17 [==============================] - 43s 3s/step - loss: 1.2181 - categorical_accuracy: 0.4530 - val_loss: 1.3475 - val_categorical_accuracy: 0.3667

Epoch 00003: saving model to model_init_2020-08-1714_29_28.508381/model-00003-1.22175-0.45232-1.34746-0.36667.h5
Epoch 4/20
17/17 [==============================] - 39s 2s/step - loss: 1.0957 - categorical_accuracy: 0.5387 - val_loss: 1.5479 - val_categorical_accuracy: 0.3500

Epoch 00004:

In [57]:
batch_size = 50

num_epochs = 5
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/5


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


17/17 [==============================] - 42s 2s/step - loss: 0.3194 - categorical_accuracy: 0.9239 - val_loss: 0.7148 - val_categorical_accuracy: 0.7167

Epoch 00001: saving model to model_init_2020-08-1714_29_28.508381/model-00001-0.31937-0.92388-0.71484-0.71667.h5
Epoch 2/5
17/17 [==============================] - 35s 2s/step - loss: 0.2728 - categorical_accuracy: 0.9481 - val_loss: 0.6098 - val_categorical_accuracy: 0.8167

Epoch 00002: saving model to model_init_2020-08-1714_29_28.508381/model-00002-0.27284-0.94810-0.60977-0.81667.h5
Epoch 3/5
17/17 [==============================] - 35s 2s/step - loss: 0.2798 - categorical_accuracy: 0.9343 - val_loss: 0.8010 - val_categorical_accuracy: 0.7167

Epoch 00003: saving model to model_init_2020-08-1714_29_28.508381/model-00003-0.27980-0.93426-0.80101-0.71667.h5
Epoch 4/5
17/17 [==============================] - 34s 2s/step - loss: 0.3027 - categorical_accuracy: 0.9204 - val_loss: 0.6790 - val_categorical_accuracy: 0.7167

Epoch 00004: sa

In [59]:
batch_size = 20

num_epochs = 30
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/30


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


17/17 [==============================] - 40s 2s/step - loss: 0.2825 - categorical_accuracy: 0.9135 - val_loss: 0.6083 - val_categorical_accuracy: 0.8000

Epoch 00001: saving model to model_init_2020-08-1714_29_28.508381/model-00001-0.28245-0.91349-0.60827-0.80000.h5
Epoch 2/30
17/17 [==============================] - 36s 2s/step - loss: 0.2857 - categorical_accuracy: 0.9343 - val_loss: 0.6752 - val_categorical_accuracy: 0.7500

Epoch 00002: saving model to model_init_2020-08-1714_29_28.508381/model-00002-0.28569-0.93426-0.67520-0.75000.h5
Epoch 3/30
17/17 [==============================] - 34s 2s/step - loss: 0.2612 - categorical_accuracy: 0.9343 - val_loss: 0.6572 - val_categorical_accuracy: 0.7667

Epoch 00003: saving model to model_init_2020-08-1714_29_28.508381/model-00003-0.26125-0.93426-0.65716-0.76667.h5

Epoch 00003: ReduceLROnPlateau reducing learning rate to 3.906250185536919e-06.
Epoch 4/30
17/17 [==============================] - 35s 2s/step - loss: 0.2768 - categorical_acc

In [60]:
batch_size = 30

num_epochs = 10
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10


/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: `imread` is deprecated!
`imread` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``imageio.imread`` instead.
  del sys.path[0]
/mnt/disks/user/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: DeprecationWarning: `imresize` is deprecated!
`imresize` is deprecated in SciPy 1.0.0, and will be removed in 1.2.0.
Use ``skimage.transform.resize`` instead.
  """


17/17 [==============================] - 42s 2s/step - loss: 0.2591 - categorical_accuracy: 0.9343 - val_loss: 0.7429 - val_categorical_accuracy: 0.7667

Epoch 00001: saving model to model_init_2020-08-1714_29_28.508381/model-00001-0.25908-0.93426-0.74292-0.76667.h5
Epoch 2/10
17/17 [==============================] - 34s 2s/step - loss: 0.2516 - categorical_accuracy: 0.9377 - val_loss: 0.5957 - val_categorical_accuracy: 0.8000

Epoch 00002: saving model to model_init_2020-08-1714_29_28.508381/model-00002-0.25159-0.93772-0.59566-0.80000.h5
Epoch 3/10
17/17 [==============================] - 35s 2s/step - loss: 0.2841 - categorical_accuracy: 0.9204 - val_loss: 0.6146 - val_categorical_accuracy: 0.7833

Epoch 00003: saving model to model_init_2020-08-1714_29_28.508381/model-00003-0.28409-0.92042-0.61456-0.78333.h5
Epoch 4/10
17/17 [==============================] - 35s 2s/step - loss: 0.2594 - categorical_accuracy: 0.9273 - val_loss: 0.6881 - val_categorical_accuracy: 0.7833

Epoch 00004: